# Assignment week 6

In this assignment we have to predict of a rating of a dress is positive or negative, based on a star rating. 1 till 3 is negative and 4 and 5 positive.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import MultinomialNB
import numpy as np

# Data pre-Processing steps

In [2]:
df = pd.read_csv('Womens Clothing E-Commerce Reviews.csv')
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [3]:
#Keep only the row with dresses and remove empty rows
df = df[(df['Class Name'] == 'Dresses')]
df.dropna(
    axis=0,
    how='any',
    thresh=None,
    subset=None,
    inplace=True
)
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
8,8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses
9,9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses
10,10,1077,53,Dress looks like it's made of cheap material,Dress runs small esp where the zipper area run...,3,0,14,General,Dresses,Dresses


In [4]:
#Make a colomn that shows of a colomn is positive or negative
def positive_or_negative(x):
    if(x > 3): 
        return 'Positive'
    else: 
        return 'Negative'

df['Rate'] = df['Rating'].apply(positive_or_negative)

#select only the needed data
df = df[["Clothing ID","Review Text", "Rating", "Class Name", "Rate"]]
df.head()


,Clothing ID,Review Text,Rating,Class Name,Rate
2,1077,I had such high hopes for this dress and reall...,3,Dresses,Negative
5,1080,"I love tracy reese dresses, but this one is no...",2,Dresses,Negative
8,1077,I love this dress. i usually get an xs but it ...,5,Dresses,Positive
9,1077,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,Dresses,Positive
10,1077,Dress runs small esp where the zipper area run...,3,Dresses,Negative


In [5]:
df['Rate'].value_counts()

Positive    4045
Negative    1326
Name: Rate, dtype: int64

# Text Modeling and Document Feature Matrix

Here we use the bag of words model we count the words and put them in bags. We are going to do so by using CountVectorizer from sklearn.feature_text_extraction library.  Since the text used is English then we are going to use _stopwords for the English language. The stop_words are informative words and frequently used in the English language and therefore will not be counted. Also, we need to convert the text to Unicode and we are going to do so by using astype('U')

In [6]:
from sklearn.feature_extraction.text import CountVectorizer #The CountVectorizer object

text = df['Review Text'].values.astype('U') #Taking the text from the df. We need to convert it to Unicode
vect = CountVectorizer(stop_words='english') #Create the CV object, with English stop words
vect = vect.fit(text) #We fit the model with the words from the review text
vect
feature_names = vect.get_feature_names() #Get the words from the vocabulary
#feature_names
print(f"There are {len(feature_names)} words in the vocabulary. A selection: {feature_names[500:520]}")


There are 7747 words in the vocabulary. A selection: ['allusion', 'allusione', 'almsot', 'alr', 'alright', 'als', 'altar', 'alter', 'alteration', 'alterations', 'altered', 'altering', 'alternate', 'alternations', 'alternative', 'althetic', 'altho', 'altogether', 'am5', 'amadi']


In [7]:
# Creates a document feature matrix
docu_feat = vect.transform(text) # make a matrix
print(docu_feat[0:500,0:500])

  (1, 7)	1
  (3, 71)	1
  (3, 214)	1
  (12, 480)	1
  (13, 200)	1
  (14, 78)	1
  (14, 408)	1
  (14, 420)	1
  (15, 37)	1
  (16, 3)	1
  (16, 44)	1
  (17, 11)	1
  (17, 236)	1
  (18, 54)	1
  (19, 39)	1
  (21, 59)	1
  (21, 216)	1
  (21, 225)	1
  (22, 229)	1
  (23, 102)	1
  (23, 178)	1
  (23, 222)	1
  (23, 243)	1
  (24, 403)	1
  (26, 11)	2
  :	:
  (475, 96)	1
  (475, 455)	2
  (476, 63)	1
  (476, 200)	1
  (476, 334)	1
  (477, 344)	1
  (478, 156)	1
  (479, 216)	1
  (479, 368)	1
  (482, 309)	2
  (484, 130)	1
  (489, 362)	1
  (490, 112)	1
  (492, 11)	2
  (492, 364)	1
  (493, 50)	1
  (493, 480)	1
  (497, 442)	1
  (498, 334)	1
  (499, 11)	1
  (499, 165)	1
  (499, 187)	1
  (499, 219)	1
  (499, 248)	1
  (499, 451)	1


# Split the file into a training and a test set

In [14]:
nb = MultinomialNB() #create the model
X = docu_feat #create the X matrix
y = df['Rate'] #create the y-variable
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) #split the data and store it



nb = nb.fit(X_train, y_train) #fit the model X=features, y=character



# Evaluate the performance of the model

In [13]:
y_test_p = nb.predict(X_test)
nb.score(X_test, y_test)

0.8542183622828784

In [9]:
from sklearn.metrics import confusion_matrix
 
cm = confusion_matrix(y_test, y_test_p)
conf_matrix = pd.DataFrame(cm, index = ["Negative (actual)", "Positive (actual)"], columns = ["Negative (predicted)", "Positive (predicted)"])
conf_matrix

,Negative (predicted),Positive (predicted)
Negative (actual),238,168
Positive (actual),67,1139


In [10]:
conf_matrix

,Negative (predicted),Positive (predicted)
Negative (actual),238,168
Positive (actual),67,1139


A calculation of the variables

$accuracy = \frac{238 + 1139}{238 + 168 + 67 + 1139} = 0.85$ The accuracy is exactly the same as the knn-score

How much of the predicted ‘negative’ is actually negative?

recall negative = $\frac{238}{238 + 168} = 0.59$

How much of the predicted ‘positive’ is actually positive?

recall positive = $\frac{1139}{1139 + 67} = 0.94$

How much of the positive is actually predict as positive?

precision positive = $\frac{1139}{1139 + 168} = 0.87$

How much of the negative is actually predict as negative?

precision negative = $\frac{238}{238 + 67} = 0.87$

In [11]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_p))

              precision    recall  f1-score   support

    Negative       0.78      0.59      0.67       406
    Positive       0.87      0.94      0.91      1206

    accuracy                           0.85      1612
   macro avg       0.83      0.77      0.79      1612
weighted avg       0.85      0.85      0.85      1612



In [12]:
df["Rate_Prediction"] = nb.predict(X)
df_contradictions = df[df["Rate"] != df["Rate_Prediction"]]
df_contradictions.head()

,Clothing ID,Review Text,Rating,Class Name,Rate,Rate_Prediction
12,1095,More and more i find myself reliant on the rev...,5,Dresses,Positive,Negative
23,1077,Cute little dress fits tts. it is a little hig...,3,Dresses,Negative,Positive
311,1089,Looks beautiful online but has too much materi...,3,Dresses,Negative,Positive
383,1104,This dress is not what i expected. the bottom ...,3,Dresses,Negative,Positive
417,1083,"I love byron lars dresses, and this design is ...",2,Dresses,Negative,Positive


In [15]:
df_contradictions["Review Text"]

12       More and more i find myself reliant on the rev...
23       Cute little dress fits tts. it is a little hig...
311      Looks beautiful online but has too much materi...
383      This dress is not what i expected. the bottom ...
417      I love byron lars dresses, and this design is ...
                               ...                        
23406    I was very excited to find a fun and lightweig...
23436    I'm 5'6" and it hits me at mid-thigh. the top ...
23475    Cute dress but not for me.  the waist is too h...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
Name: Review Text, Length: 506, dtype: object

Now you can see whitch reviews are wrong predicted